<a href="https://colab.research.google.com/github/HakureiPOI/llm-stock-rec/blob/main/notebook/tushare_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tushare -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 3.2 MB/s eta 0:00:00


In [3]:
import tushare as ts
from typing import List, Optional
# from config import settings
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [4]:
class TushareService:
    def __init__(self):
        self.pro = None
        self._init_tushare()

    def _init_tushare(self):
        """初始化 Tushare"""
        # if not settings.TUSHARE_TOKEN:
        #     raise ValueError("TUSHARE_TOKEN 环境变量未设置，请在 .env 文件中配置")

        # ts.set_token(settings.TUSHARE_TOKEN)
        ts.set_token('嘻嘻不给你看')
        self.pro = ts.pro_api()

    def get_stock_info(self, ts_codes: List[str]) -> List[dict]:
        try:
            ts_code_str = ','.join(ts_codes)
            df = self.pro.stock_basic(ts_code=ts_code_str)
            return df.to_dict('records')
        except Exception as e:
            raise Exception(f"获取股票基本信息失败: {str(e)}")

    def get_realtime_quote(self, ts_codes : List[str]) -> List[dict]:
        try:
            ts_code_str = ','.join(ts_codes)
            df = self.pro.daily(ts_code=ts_code_str)

            if df.empty:
                return []

            # 按交易日期降序排列，取每个股票的最新一条
            df = df.sort_values('trade_date', ascending=False)
            df = df.drop_duplicates('ts_code', keep='first')

            return df.to_dict('records')
        except Exception as e:
            raise Exception(f"获取实时行情失败: {str(e)}")

    def get_historical_data(self, ts_codes : List[str], start_date : Optional[str] = None, end_date : Optional[str] = None) -> List[dict]:
        try:
            # If dates are not provided, calculate and format them for Tushare API (YYYYMMDD)
            if not end_date:
                end_date_dt = datetime.now()
                end_date = end_date_dt.strftime('%Y%m%d')

            if not start_date:
                start_date_dt = (datetime.now() - timedelta(days=365))
                start_date = start_date_dt.strftime('%Y%m%d')

            ts_code_str = ','.join(ts_codes)

            df = self.pro.daily(
                ts_code=ts_code_str,
                start_date=start_date,
                end_date=end_date,
            )

            if df.empty:
                return []

            df.sort_values('trade_date', inplace=True)

            return df.to_dict('records')
        except Exception as e:
            raise Exception(f"获取历史数据失败: {str(e)}")


In [5]:
def test_tushare_service():
    print("=== 开始测试 TushareService ===")
    try:
        # 1. 初始化服务
        service = TushareService()
        print("服务初始化成功")

        # 测试用的股票代码 (平安银行, 浦发银行)
        test_codes = ['000001.SZ', '600000.SH']

        # 2. 测试获取股票基本信息
        print(f"\n[测试 1] 获取股票基本信息: {test_codes}")
        stock_infos = service.get_stock_info(test_codes)
        print(f"获取到 {len(stock_infos)} 条记录")
        if stock_infos:
            print(f"示例数据: {stock_infos[0].get('name')} ({stock_infos[0].get('ts_code')})")

        # 3. 测试获取实时行情 (最新日线数据)
        print(f"\n[测试 2] 获取最新行情: {test_codes}")
        quotes = service.get_realtime_quote(test_codes)
        print(f"获取到 {len(quotes)} 条记录")
        if quotes:
            print(f"示例数据: 日期 {quotes[0].get('trade_date')}, 收盘价 {quotes[0].get('close')}")

        # 4. 测试获取历史数据
        print(f"\n[测试 3] 获取历史数据 (默认最近一年)")
        history = service.get_historical_data(test_codes)
        print(f"获取到 {len(history)} 条记录")
        if history:
            print(f"最早一条: {history[0].get('trade_date')}")
            print(f"最新一条: {history[-1].get('trade_date')}")

        print("\n=== 测试完成 ===")

    except Exception as e:
        print(f"\n!!! 测试过程中发生错误: {e}")
        print("请检查 TUSHARE_TOKEN 是否正确设置。")

if __name__ == '__main__':
    test_tushare_service()

=== 开始测试 TushareService ===
服务初始化成功

[测试 1] 获取股票基本信息: ['000001.SZ', '600000.SH']
获取到 2 条记录
示例数据: 平安银行 (000001.SZ)

[测试 2] 获取最新行情: ['000001.SZ', '600000.SH']
获取到 2 条记录
示例数据: 日期 20260128, 收盘价 10.84

[测试 3] 获取历史数据 (默认最近一年)
获取到 486 条记录
最早一条: 20250205
最新一条: 20260128

=== 测试完成 ===
